# Testing

In this notebook you can explore and test the resulting database tables from our Dagster ETL process.
Here you can analyze


In [2]:
import duckdb
import polars as pl
from IPython import display

### Create Database Connection


In [3]:
%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/storage/data-system-project.db")
%sql conn --alias duckdb
%sql SHOW ALL TABLES;

Tip: You may define configurations in /home/workspaces/DSP-compose/pyproject.toml or /root/.jupysql/config.

Running query in 'duckdb'

database,schema,name,column_names,column_types,temporary
data-system-project,cleaned,cleaned_knmi_weather_data,"['Station_code', 'Date', 'Hour', 'Dd', 'Fh', 'Ff', 'Fx', 'T', 'T10n', 'Td', 'Sq', 'Q', 'Dr', 'Rh', 'P', 'Vv', 'N', 'U', 'Ww', 'Ix', 'M', 'R', 'S', 'O', 'Y']","['BIGINT', 'DATE', 'TINYINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT']",False
data-system-project,cleaned,cleaned_storm_incidents,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT']",False
data-system-project,joined,incident_deployments,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
data-system-project,joined,incident_deployments_vehicles,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident', 'Vehicle']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
data-system-project,joined,incident_deployments_vehicles_weather,"['Station_code', 'Date', 'Hour', 'Dd', 'Fh', 'Ff', 'Fx', 'T', 'T10n', 'Td', 'Sq', 'Q', 'Dr', 'Rh', 'P', 'Vv', 'N', 'U', 'Ww', 'Ix', 'M', 'R', 'S', 'O', 'Y', 'Incident_ID', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident', 'Vehicle']","['BIGINT', 'DATE', 'TINYINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
data-system-project,joined,incident_deployments_vehicles_wijken,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'I

##### Drop Tables


In [4]:
# %sql DROP TABLE public.cbs_buurten;
# %sql DROP TABLE public.cbs_wijken;

##### Retrieve Tables as Polars DataFrame


In [5]:
cbs_wijken = conn.execute(
    """
    SELECT * FROM public.cbs_wijken """
).pl()

service_areas = conn.execute(
    """
    SELECT * FROM public.service_areas """
).pl()

incident_deployments_vehicles = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles"""
).pl()


cleaned_knmi_weather_data = conn.execute(
    """
    SELECT * FROM cleaned.cleaned_knmi_weather_data
    """
).pl()


incident_deployments_vehicles_wijken = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles_wijken
    """
).pl()

incident_deployments_vehicles_weather = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles_weather
    """
).pl()


# Close the database connection
conn.close()

In [6]:
# cbs_wijken.filter(
#     [
#         pl.col("gemeentenaam") == "Amsterdam",
#     ]
# ).head()

In [7]:
# incident_deployments_vehicles_wijken.head()

In [8]:
incident_deployments_vehicles_weather.head()

Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,Sq,Q,Dr,Rh,P,Vv,N,U,Ww,Ix,M,R,S,O,Y,Incident_ID,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT,Incident_Endtime_Hour,Incident_Duration_Hour,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
i64,date,i8,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,time,time,time,i64,str,str,str,f64,f64,i8,i8,i8,i8,i8,i64,str,str,str,str,str,str
240,2005-01-01,1,260,40,30,60,68,null,57,0,0,0,0,10246,57,8,93,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,2,230,30,30,60,65,null,52,0,0,0,0,10244,58,8,91,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,3,230,40,30,50,43,null,34,0,0,0,0,10241,40,1,94,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,4,220,40,40,50,38,null,32,0,0,0,0,10239,12,0,96,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,5,230,40,40,50,38,null,34,0,0,0,0,10237,14,3,97,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [10]:
# Select all rows where Incidents happened (by filtering on Incident_ID != null)
selected_df_2005 = incident_deployments_vehicles_weather.filter(
    pl.col("Incident_ID").is_not_null(), pl.col("Date").dt.year() == 2005
)
display(selected_df_2005.head())
print(selected_df_2005.columns)

Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,Sq,Q,Dr,Rh,P,Vv,N,U,Ww,Ix,M,R,S,O,Y,Incident_ID,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT,Incident_Endtime_Hour,Incident_Duration_Hour,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
i64,date,i8,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,time,time,time,i64,str,str,str,f64,f64,i8,i8,i8,i8,i8,i64,str,str,str,str,str,str
240,2005-04-07,12,240,160,160,210,99,69,13,8,195,0,0,9993,74,6,55,2,7,0,0,0,0,0,11333,12:09:51,13:38:19,01:28:28,3,"""IJsbrand""","""AMSTERDAM""","""Unknown""",4.887098,52.418767,13,1,9,38,28,94,"""TS""","""TS""","""IJsbrand""","""Professional""","""NULL""","""Tankautospuit"""
240,2005-04-07,12,240,160,160,210,99,69,13,8,195,0,0,9993,74,6,55,2,7,0,0,0,0,0,11333,12:09:51,13:38:19,01:28:28,3,"""IJsbrand""","""AMSTERDAM""","""Unknown""",4.887098,52.418767,13,1,9,38,28,95,"""TS""","""TS""","""Zebra""","""Professional""","""808""","""Tankautospuit"""
240,2005-04-07,12,240,160,160,210,99,69,13,8,195,0,0,9993,74,6,55,2,7,0,0,0,0,0,11337,12:53:52,13:57:46,01:03:54,2,"""Pieter""","""AMSTERDAM""","""Tree""",4.826198,52.352551,13,1,53,57,3,97,"""RV""","""RV""","""Teunis""","""Professional""","""753""","""HV met kraan"""
240,2005-04-07,12,240,160,160,210,99,69,13,8,195,0,0,9993,74,6,55,2,7,0,0,0,0,0,11337,12:53:52,13:57:46,01:03:54,2,"""Pieter""","""AMSTERDAM""","""Tree""",4.826198,52.352551,13,1,53,57,3,98,"""TS""","""TS""","""Osdorp""","""Professional""","""633""","""Tankautospuit"""
240,2005-04-07,12,240,160,160,210,99,69,13,8,195,0,0,9993,74,6,55,2,7,0,0,0,0,0,11337,12:53:52,13:57:46,01:03:54,2,"""Pieter""","""AMSTERDAM""","""Tree""",4.826198,52.352551,13,1,53,57,3,98,"""TS""","""TS""","""Osdorp""","""Professional""","""633""","""Tankautospuit"""


['Station_code', 'Date', 'Hour', 'Dd', 'Fh', 'Ff', 'Fx', 'T', 'T10n', 'Td', 'Sq', 'Q', 'Dr', 'Rh', 'P', 'Vv', 'N', 'U', 'Ww', 'Ix', 'M', 'R', 'S', 'O', 'Y', 'Incident_ID', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident', 'Vehicle']


In [11]:
incident_deployments_vehicles_weather.head()

Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,Sq,Q,Dr,Rh,P,Vv,N,U,Ww,Ix,M,R,S,O,Y,Incident_ID,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT,Incident_Endtime_Hour,Incident_Duration_Hour,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
i64,date,i8,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,time,time,time,i64,str,str,str,f64,f64,i8,i8,i8,i8,i8,i64,str,str,str,str,str,str
240,2005-01-01,1,260,40,30,60,68,null,57,0,0,0,0,10246,57,8,93,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,2,230,30,30,60,65,null,52,0,0,0,0,10244,58,8,91,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,3,230,40,30,50,43,null,34,0,0,0,0,10241,40,1,94,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,4,220,40,40,50,38,null,32,0,0,0,0,10239,12,0,96,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,5,230,40,40,50,38,null,34,0,0,0,0,10237,14,3,97,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
# Feature engineering (if needed)

# Encoding categorical variables (if any)

# Prepare your data
# X = combined_df.drop(
#     "incident_column", axis=1
# )  # Replace 'incident_column' with your target column name
# y = combined_df["incident_column"]

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# # Train XGBoost model
# model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
# model.fit(X_train, y_train)

# # Evaluate the model
# predictions = model.predict(X_test)
# accuracy = accuracy_score(y_test, predictions)
# print(f"Model Accuracy: {accuracy}")

# # Feature Importance
# feature_importances = model.feature_importances_
# features = X.columns
# importance_df = pd.DataFrame({"Feature": features, "Importance": feature_importances})
# importance_df = importance_df.sort_values(by="Importance", ascending=False)
# print(importance_df)